# Imports

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud.bigquery import magics
import google.auth
from numpy.core.arrayprint import DatetimeFormat
credentials, project = google.auth.default()

In [ ]:
!pip3 install -r /content/drive/MyDrive/workspace/cur-sol/train/requirements.txt


     |████████████████████████████████| 276 kB 5.0 MB/s 
     |████████████████████████████████| 18.6 MB 151 kB/s 
     |████████████████████████████████| 552 kB 33.7 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 


In [ ]:
from deepchem.models.keras_model import KerasModel
import deepchem as dc
import tensorflow as tf
print(f'dc: {dc.__version__}')
print(f'tf: {tf.__version__}')


dc: 2.5.0
tf: 2.6.0


In [ ]:
tasks, datasets, transformers = dc.molnet.load_bace_classification(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

In [ ]:
from deepchem.models.graph_models import GraphConvModel
model_dir = 'gs://cur-solubility/models/'
model = GraphConvModel(model_dir=model_dir, n_tasks=1, mode='regression', dropout=0.2)
model.fit(train_dataset, nb_epoch=15, deterministic=True)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(780,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(780, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_17:0", shape=(2600,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_16

0.7201420031095805

In [ ]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print('training set score:', model.evaluate(train_dataset, [metric]))apt update
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'pearson_r2_score': 0.09818907232128887}
test set score: {'pearson_r2_score': 0.0037047521626813267}


In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)
model_dir = 'gs://cur-solubility/models/'
latest = tf.train.latest_checkpoint(model_dir)
latest

2.6.0


'gs://cur-solubility/models/ckpt-1'

In [ ]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'pearson_r2_score': 0.09818907232128887}
test set score: {'pearson_r2_score': 0.0037047521626813267}


In [ ]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

In [ ]:
model = create_model()
model.load_weights(latest)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Get Docker!

In [ ]:
!apt-get -qq install docker.io

In [ ]:
%%shell
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker info
docker network ls
docker pull hello-world
docker pull ubuntu
# docker build -t myimage .
docker images


GCP_PROJECT_ID = 'aburdenko-project'
PROJECT_NUMBER = '653488387759'
SERVICE_ACCOUNT_NAME ='aburdenko-jupyter-notebook'
ACCOUNT = f'{SERVICE_ACCOUNT_NAME}@{GCP_PROJECT_ID}.iam.gserviceaccount.com'
gcloud auth activate-service-account --key-file=/content/drive/MyDrive/workspace/aburdenko-project-ec35c91e6a60.json 
gcloud auth configure-docker 
gcloud beta ai custom-jobs local-run \
  --base-image=gcr.io/aburdenko-project/deepchem \
  --work-dir=/content/drive/MyDrive/workspace/cur-sol/train \
  --script=trainer.py \
  --output-image-uri=gcr.io/aburdenko-project/deepchem 

kill $(jobs -p)



+ dockerd -b none --iptables=0 -l warn
++ seq 5
+ for i in $(seq 5)
+ '[' '!' -S /var/run/docker.sock ']'
+ sleep 2
WARN[0000] deprecated version : `1`, please switch to version `2` 
WARN[2021-08-25T01:37:48.859859666Z] failed to load plugin io.containerd.snapshotter.v1.devmapper  error="devmapper not configured"
WARN[2021-08-25T01:37:48.860624485Z] could not use snapshotter devmapper in metadata plugin  error="devmapper not configured"
ERRO[2021-08-25T01:37:48.899813239Z] failed to mount overlay: invalid argument     storage-driver=overlay2
ERRO[2021-08-25T01:37:48.899978617Z] exec: "fuse-overlayfs": executable file not found in $PATH  storage-driver=fuse-overlayfs
ERRO[2021-08-25T01:37:48.904804849Z] AUFS was not found in /proc/filesystems       storage-driver=aufs
ERRO[2021-08-25T01:37:48.917423128Z] failed to mount overlay: invalid argument     storage-driver=overlay
WARN[2021-08-25T01:37:48.933073120Z] Your kernel does not support cgroup blkio weight 
WARN[2021-08-25T01:37:48.9332